In [ ]:
from utils import r14, r32, r50, r125, r134a, r143a, r170, atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real
from fffit.fffit.plot import plot_obj_contour

In [ ]:
import signac
from pathlib import Path
import os
import json
import flow
from flow import FlowProject, directives

#Import dependencies
import numpy as np
import templates.ndcrc
from utils import r14, r32, r50, r125, r134a, r143a, r170, atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real
from fffit.fffit.plot import plot_obj_contour
import pickle
import gzip

#Define method, ep_enum classes, indecies to consider, and kernel
training_molecules = ["R14"]

#Set params for saving results, # of repeats, and the seed
obj_choice = "ExpVal"
total_repeats  = 30
repeat_num = 1
seed = 1
save_data = True
at_class = atom_type.make_atom_type_class(9)

#Load class properies for each trainingmolecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

#Get dict of refrigerant classes to consider, gps, and atom typing class
all_molec_data_dict = {"R14":r14_class, 
                "R32":r32_class, 
                "R50":r50_class, 
                "R170":r170_class, 
                "R125":r125_class, 
                "R134a":r134a_class, 
                "R143a":r143a_class}

#Make molec data dict given molecules:
molec_data_dict = {}
for molec in training_molecules:
    if molec in list(all_molec_data_dict.keys()):
        molec_data_dict[molec] = all_molec_data_dict[molec]

all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
driver = opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, total_repeats, seed, obj_choice)

#Create param sets for the AT optimization based on seed and such
param_inits = driver.get_param_inits()
#Get the correct parameter set from the param_inits based on which repeat we are evaluating
param_guess = param_inits[repeat_num-1].reshape(1,-1)
#Optimize atom types
ls_results, sort_ls_res, best_runs = driver.optimize_ats(param_guess, repeat_num-1)

if save_data == True:
    #Save results for best set for each run and iter to a csv file in Results
    #Ensure directory exists
    dir_name = driver.make_results_dir(training_molecules)
    os.makedirs(dir_name, exist_ok=True) 
    save_path3 = os.path.join(dir_name, "best_per_run.csv")
    #Save results. Append to file if it already exists
    if os.path.exists(save_path3):
        best_runs.to_csv(save_path3, mode = "a", index = False, header = False)
    else:
        best_runs.to_csv(save_path3, index = False)

In [ ]:
#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

In [ ]:
#Get Setup properties
at_class = atom_type.AT_Scheme_10()
w_calc = 2
save_data = False
obj_choice = "ExpVal"
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
setup = opt_atom_types.Problem_Setup(molec_data_dict, all_gp_dict, at_class, w_calc, obj_choice, save_data)


In [ ]:
#Visualize property predictions for training data
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, w_calc, obj_choice, save_data)
molec_names = list(["R14", "R32", "R50", "R170", "R125", "R134a", "R143a"])
visual.comp_paper_full_ind(molec_names)
#Calculate MAPD for predictions
df = visual.calc_MAPD_best(molec_names)
df

In [ ]:
#Sort MAPD values from above df
filtered_df = df[df['Model'] == 'Opt R14-R32-R50-R170-R125-R134a-R143a']
# filtered_df = df[df['Model'] != 'Literature']
sorted_df = filtered_df.sort_values(by='MAPD', ascending = False).reset_index(drop = True)
sorted_df
# average_df = filtered_df.groupby('Molecule')['MAPD'].mean().reset_index()
# average_df.rename(columns={'MAPD': 'Avg MAPD'}, inplace=True)
# average_df
# avg_df = average_df.sort_values(by='Avg MAPD', ascending = True).reset_index(drop = True)
# avg_df
# df.to_csv("MAPD_best.csv", index = False, header = True)

In [ ]:
import scipy 
import os
repeats = 1
seed = 1
#Get obj from a set of parameters
molec_data_dict = {"R14":r14_class, "R32":r32_class}#, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
# molec_data_dict = {"R32":r32_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, obj_choice)
best_pref = np.array([3.3965664266801703,3.440648986884316,2.0002287496346898,1.9895617598434474,2.8455549929882973,2.1846771895375956,2.929597585178781,42.15585471681824,35.789885824993554,45.023087710218206,10.000000000000002,42.82067021601047,22.155828740603113,28.599458026860685])
# best_pref = np.array([3.563847649565008, 3.7023297804578013, 3.383454561616988, 2.501869323173524, 2.8016532687774456, 2.9570560025914276, 2.938081818350493, 47.955100549476825, 42.9383601572982, 51.7928754827407, 9.956251656605064, 39.956127381108494, 28.664773473001603, 24.34860285294489]
# )
# best_pref = np.array([3.727,3.810,3.754, 3.028,3.809,
#                       2.316,
#                       2.607,3.255,3.424,2.917,
#                       71.730,52.200, 72.610,53.690,69.970,
#                       6.160,
#                       39.980,20.164, 17.050, 29.070])
best_real= at_optimizer.values_pref_to_real(best_pref)
obj = at_optimizer.calc_obj(best_real)
print(obj[0])

#Get Jac and Hess info from a set of parameters
# dir_name = at_optimizer.make_results_dir(list(at_optimizer.molec_data_dict.keys()))
# save_path = os.path.join(dir_name, "best_set" + "_hess_approx.npy")
# hess = np.load(save_path)
# jac = at_optimizer.approx_jac(best_real)
# hess = at_optimizer.approx_hess(best_real)
# print(scipy.linalg.norm(jac, ord = np.inf))
# print(jac)
# print(hess)
# eigval, eigvec = scipy.linalg.eig(hess)
# print(eigval)

In [ ]:
#Check progress of indivisual parameter sets
import os
import tensorflow as tf
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, w_calc, obj_choice, save_data)
for molec_names in list(molec_data_dict.keys()):
    param_matrix = visual.at_class.get_transformation_matrix({molec_names: molec_data_dict[molec_names]})
    #Get best_per_run.csv for all molecules
    molec_dir = visual.make_results_dir([molec_names])
    if os.path.exists(molec_dir+"/best_per_run.csv"):
        print(molec_dir)
        molec_df = pd.read_csv(molec_dir+"/best_per_run.csv", header = 0)
        # print(molec_df.head())
        first_param_name = at_class.at_names[0] + "_min"
        last_param_name = at_class.at_names[-1] + "_min"
        print(molec_df.loc[0, "Min Obj"])
        molec_best = molec_df.loc[0, first_param_name:last_param_name].values
        print(molec_best)
        # ind_best_real = visual.values_pref_to_real(molec_best)
        # ind_best_scl = values_real_to_scaled(ind_best_real.reshape(1,-1), at_class.at_bounds_nm_kjmol)
        ind_best_gp = molec_best.reshape(-1,1).T@param_matrix
        # ind_best_gp = tf.convert_to_tensor(ind_best_gp, dtype=tf.float64)
        print(ind_best_gp)

In [ ]:
#Get Uncertainty data
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
df = pd.DataFrame(columns = ["Molecule", "Property", "Value", "Uncertainty", "Uncertainty Used", "Weight"])
for key in list(molec_data_dict.keys()):
    molec_object = molec_data_dict[key]
    for prop_key in list(molec_object.uncertainties.keys()):
        if "vap_density" in prop_key:
            exp_data = molec_object.expt_vap_density
            property_bounds = molec_object.vap_density_bounds
            property_name = "Vapor Density [kg/m^3]"
        elif "liq_density" in prop_key:
            exp_data = molec_object.expt_liq_density
            property_bounds = molec_object.liq_density_bounds
            property_name = "Liquid Density [kg/m^3]"
        elif "Pvap" in prop_key: 
            exp_data = molec_object.expt_Pvap
            property_bounds = molec_object.Pvap_bounds
            property_name = "Vapor pressure [bar]"
        elif "Hvap" in prop_key:
            exp_data = molec_object.expt_Hvap
            property_bounds = molec_object.Hvap_bounds
            property_name = "Enthalpy of Vaporization [kJ/kg]"

        values = np.array(list(exp_data.values()))
        y_unc = molec_object.uncertainties[prop_key]
        unc_val = np.maximum(y_unc, 0.02)
        weight = 1/(unc_val*values)**2
        df_vals = [key, prop_key, values, y_unc, unc_val, weight]
        iter_df = pd.DataFrame([df_vals], columns = df.columns)
        iter_df = iter_df.apply(lambda col: col.explode(), axis=0).reset_index(drop=True).copy(deep =True)
        df = pd.concat([df, iter_df], ignore_index=True)
df.to_csv("molecule_exp_unc_data.csv", index = False, header = True)

In [ ]:
#Get best values from csvs
def get_best_results(visual, molec_names):
    best_vals = {}
    #Get best_per_run.csv for all molecules
    all_molec_dir = visual.make_results_dir(molec_names)
    all_df = pd.read_csv(all_molec_dir+"/best_per_run.csv", header = 0)
    # print(all_df)
    first_param_name = visual.at_class.at_names[0] + "_min"
    last_param_name = visual.at_class.at_names[-1] + "_min"
    full_opt_best = all_df.loc[0, first_param_name:last_param_name].values
    best_vals["All"] = full_opt_best
    for i in range(len(molec_names)):
        molec_dir = visual.make_results_dir([molec_names[i]])
        molec_df = pd.read_csv(molec_dir+"/best_per_run.csv", header = 0)
        molec_best = molec_df.loc[0, first_param_name:last_param_name].values
        best_vals[molec_names[i]] = molec_best
    return best_vals